<a href="https://www.kaggle.com/code/volt3000/fine-tune-llama-3-instruct-8b-on-codesearchnet-alp?scriptVersionId=189927529" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Fine-tune Llama-3-8B-Instuct with Unsloth on CodeSearchNet

> Note: This notebooks runs best when it's accelerated with Nvidia T4(s) or GPU(s) of similar architecture 

## Download, Install and Import Dependencies

In [ ]:
!mamba install --quiet --force-reinstall aiohttp -y
!pip install -qU "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install -q "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install wandb evaluate accelerate

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

import os
os.environ["WANDB_DISABLED"] = "true"

warning  libmamba Cache file "/opt/conda/pkgs/cache/c6f2354e.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/86b0f08d.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/c9ddbd6b.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/b121c3e7.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/497deca9.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/09cdf8bf.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/47929eba.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/3e39a7aa.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/2ce54b42.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/4ea078d6.json" was modified by another program


In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import pprint as pp
from datasets import load_dataset
import torch

## Setup Model and Tokenizer from Unsloth

In [ ]:
max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True, # Set to True if out of memory (default is "unsloth")
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

## Format CodeSearchNet to Alpaca-styled Format

In [ ]:
alpacaFormatString = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN (<|eot_id|>)

# Define the formatting function to initially drop all the unnecessary columns and rename what we need
def formatFunctionSample(sample):
    language = sample['language']
    instruction = f"What does this {language} function do?"
    inputText = sample['func_code_string']
    outputText = sample['func_documentation_string']

    # Returning a dictionary of the necessary columns
    return {
        "instruction": instruction,
        "input": inputText,
        "output": outputText
    }

# Define the function to create the new 'text' column
def createAlpacaFormatString(sample):
    instruction = sample['instruction']
    inputText = sample['input']
    outputText = sample['output']
    
    text = alpacaFormatString.format(instruction, inputText, outputText) + EOS_TOKEN
    sample['text'] = text
    
    return sample

In [ ]:
dataset = load_dataset("claudios/code_search_net", "python", split="train[:21]")

# Mapping the existing dataset to the new format keeping only the keys of the dictionary we returned
dataset = dataset.map(formatFunctionSample, remove_columns=dataset.column_names)

# Adding the text column to the new dataset
dataset = dataset.map(createAlpacaFormatString)

In [ ]:
dataset

In [ ]:
pp.pp(dataset[0])

In [ ]:
print(dataset[0]['text'])

## Train-test Split

In [ ]:
datasetDictionary = dataset.train_test_split(test_size=0.33) 

In [ ]:
datasetDictionary

## Initialize Trainer with Training Arguments

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasetDictionary["train"],
    eval_dataset = datasetDictionary["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs = 10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 8402,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

## Fine-tune Training Loop

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Run Inference on Fine-tuned Model

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

testFunction = """
def find_prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors
"""

testDocstring = """
Finds all prime factors of the given integer n and returns them as a list.

Parameters:
n (int): The integer to find the prime factors of.

Returns:
list: A list containing all prime factors of n.
"""

inputs = tokenizer(
[
    alpacaFormatString.format(
        "What does this python function do?", # instruction
        testFunction, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)